# Entraîner un étiqueteur grammatical basé sur le *French Treebank*

## Un corpus arboré pour le français

Le [*French Treebank*](http://ftb.linguist.univ-paris-diderot.fr/), développé à l’Université de Paris depuis 1997, est un corpus annoté (annotations lexicales et syntaxiques) à partir des articles du journal *Le Monde* pour la période 1990-1993. Plus-value indéniable : toutes les annotations ont été validées à la main.

Il totalise, dans sa version 1.0, 21 550 phrases pour 664 500 tokens.

Un extrait de 2072 phrases (pour 58 527 tokens), dans une version appauvrie au format *word/tag*, est disponible dans le dossier *./ftb*.

Pour charger les quatre fichiers de cet extrait :

In [ ]:
from nltk.corpus.reader import TaggedCorpusReader

reader = TaggedCorpusReader('./data/ftb', r'.*\.pos')

## Entraîner un étiqueteur pour unigrammes

Ainsi outillé par NLTK, le corpus est facilement exploitable pour entraîner un étiqueteur pour unigrammes :

In [ ]:
from nltk.tag import UnigramTagger

train = reader.tagged_sents()
tagger = UnigramTagger(train)

Considérons trois phrases que nous souhaitons étiqueter avec ce modèle entraîné sur le FTB. La première phrase est issue de ce modèle, la seconde est extraite de la version complète du FTB mais ne figure pas dans le modèle, et quant à la troisième, elle est purement fictive :

In [ ]:
sents = [
    ['La', 'tâche', 'des', 'secouristes', 'est', 'immense', ',', 'faute', 'de', 'moyens', 'matériels', 'et', 'humains', '.'],
    ['Elle', 'ne', 'sera', 'vaincue', 'que', 'grâce', 'à', 'une', 'alliance', 'franco', '-', 'allemande', 'âprement', 'négociée', '.'],
    ['Julien', 'est', 'tombé', 'lourdement', 'de', 'sa', 'chaise', '.']
]

Nous connaissons l’étiquetage grammatical des deux premières et soumettons celui de la troisième :
```txt
(1) La/D tâche/N des/P+D secouristes/N est/V immense/A ,/PONCT faute/N de/P moyens/N matériels/A et/C humains/A ./PONCT
(2) Elle/CL ne/ADV sera/V vaincue/V que/ADV grâce/N à/P une/D alliance/N franco/A -/PONCT allemande/A âprement/ADV négociée/V ./PONCT
(3) Julien/N est/V tombé/V lourdement/ADV de/P sa/D chaise/N ./PONCT
```

Pour lancer l’étiquetage automatique, l’interface `UnigramTagger` fournit deux méthodes :
- `.tag()` pour une seule phrase ;
- `.tag_sents()` pour une liste de phrases.

In [ ]:
sents_tagged = tagger.tag_sents(sents)

Comparons le résultat de l’étiquetage automatique pour la première phrase avec la solution connue :

In [ ]:
[ tup for tup in sents_tagged[0] ]

Aucune erreur, le résultat est pleinement satisfaisant. Ce n’est pas une surprise, la phrase étiquetée fait partie du modèle.

Pour la seconde phrase, issue du même corpus bien qu’en dehors du modèle, le traitement lève des imprécisions :

In [ ]:
[ tup for tup in sents_tagged[1] ]

Les différences à noter sont :
- *vaincue* : `None` au lieu de `V`
- *que* : `C` au lieu de `ADV`
- *âprement* : `None` au lieu de `ADV`

Sur 15 tokens, 3 différences, soit un taux d’erreur de 20 % !

Pour la dernière phrase, le taux d’erreur (37,5 %) est encore plus prononcé :

In [ ]:
[ tup for tup in sents_tagged[2] ]

**Rappel :** plus le corpus d’entraînement est large, meilleur sera l’étiquetage. Ici, l’extrait se contente de 1/10e du FTB. Avec la totalité du corpus, il est fort probable que la plupart des erreurs disparaîtraient.

## Évaluer la performance d’un étiqueteur

Afin d’évaluer rapidement la justesse de l’étiqueteur, NLTK met à disposition une méthode qui nécessite de diviser les données en deux jeux :
- un jeu pour l’entraînement de l’étiqueteur ;
- un jeu pour le tester.

Un taux de 80/20 est adapté pour effectuer cette évaluation :

In [ ]:
nb_sents = len(reader.tagged_sents())
limit = int(nb_sents * 0.2)

# split
train_data = reader.tagged_sents()[limit:]
test_data = reader.tagged_sents()[:limit]

Il ne reste plus qu’à entraîner l’étiqueteur puis à l’évaluer avec les données de test. La métrique utilisée ci-dessous est l’exactitude :

In [ ]:
tagger = UnigramTagger(train_data)
tagger.accuracy(test_data)

Face à une ambiguïté (*que* : `C` ou `ADV` ?), un étiqueteur décide en fonction du contexte grâce à un score de fréquence d’occurrences. Le paramètre `cutoff` fixe un seuil minimal avant d’attribuer une étiquette, au prix d’une diminution des performances globales :

In [ ]:
tagger = UnigramTagger(train_data, cutoff=3)
tagger.accuracy(test_data)

Un taux d’exactitude de 82 % pour un jeu de données si réduit se révèle être un bon score, même s’il n’est pas suffisant. À titre de comparaison avec un extrait de même volume du corpus *Brown*, le taux d’exactitude est seulement de 77 % :

In [ ]:
from nltk.corpus import brown

subcorpus = brown.tagged_sents()[:2071]
limit = int(len(subcorpus) * 0.2)

train_data = subcorpus[limit:]
test_data = subcorpus[:limit]

tagger = UnigramTagger(train_data)
tagger.accuracy(test_data)

D’autres mesures quantitatives (*precision*, *recall* et *f1-score*) apporteraient plus de finesse à l’évaluation d’un étiqueteur. Elles seront abordées dans un autre chapitre.

**Attention :** le test de performance ne juge pas la qualité de l’annotation produite mais celle de l’étiqueteur.

## Améliorer la performance d’un étiqueteur

La première piste pour améliorer la performance d’un étiqueteur est de décider, lorsqu’il est face à un contexte inconnu, d’une étiquette par défaut. C’est ce que permet la classe `DefaultTagger` :

In [ ]:
from nltk.tag import DefaultTagger

default = DefaultTagger('N')
tagger = UnigramTagger(train_data, backoff=default)
tagger.accuracy(test_data)

Une amélioration nette de presque 8 % avec cette simple astuce !

La seconde piste, qui découle de la première, consiste à entraîner puis à combiner des étiqueteurs pour *n*-grammes. NLTK fournit les classes `BigramTagger` et `TrigramTagger`.

Isolément, leurs performances sont très mauvaises :

In [ ]:
from nltk.tag import BigramTagger

bigram_tagger = BigramTagger(train_data)
bigram_tagger.accuracy(test_data)

In [ ]:
from nltk.tag import TrigramTagger

trigram_tagger = TrigramTagger(train_data)
trigram_tagger.accuracy(test_data)

La solution consiste à entraîner chaque étiqueteur avec le précédent !

In [ ]:
bigram_tagger = BigramTagger(train_data, backoff=tagger)
trigram_tagger = TrigramTagger(train_data, backoff=bigram_tagger)

Le résultat des évaluations montre que l’étiqueteur 2-grammes est le plus performant :

In [ ]:
bigram_accuracy = bigram_tagger.accuracy(test_data)
trigram_accuracy = trigram_tagger.accuracy(test_data)

print(
    f"Évaluation de l’étiqueteur 2-grammes : { bigram_accuracy }",
    f"Évaluation de l’étiqueteur 3-grammes : { trigram_accuracy }",
    sep="\n"
)

## Gérer la sérialisation d’un étiqueteur

Par *sérialisation*, on entend une opération de codage de l’étiqueteur sous une forme allégée, à fin notamment de sauvegarde.

Le module `pickle` de Python permet ainsi de sauvegarder la configuration d’un étiqueteur entraîné pour la charger plus tard.

### Sérialiser un objet

Importer le module `pickle` puis appeler la méthode `.dump()` sur une ressource de fichier en veillant à ce qu’elle soit disponible en écriture en mode binaire.

In [ ]:
import pickle

with open('./data/ftb/tagger.pickle', 'wb') as dest:
    pickle.dump(bigram_tagger, dest)

### Charger un objet sérialisé

Importer le module `pickle` puis appeler la méthode `.load()` sur une ressource de fichier en veillant à ce qu’elle soit disponible en lecture en mode binaire.

In [ ]:
import pickle

with open('./data/ftb/tagger.pickle', 'rb') as f:
    tagger = pickle.load(f)

Une fois l’étiqueteur chargé, on peut l’utiliser normalement :

In [ ]:
sents_tagged = tagger.tag_sents(sents)